# Sentdex video
https://www.youtube.com/watch?v=NwH0HvMI4EA&t=539s

In [3]:
# 10 threads, 20 Jobs. Each Job takes 0.5 seconds
# Normally, should've taken 10 secs total
# But it actually took very less.

import threading
from queue import Queue
import time # to mimic processing time only. so just time.sleep()

print_lock = threading.Lock() # to lock printing

def exampleJob(worker):
    # mimics computing time
    time.sleep(0.5)
    
    with print_lock:
        print(threading.current_thread().name, worker)

def threader():
    while True:
        worker = q.get()
        exampleJob(worker)
        q.task_done() # release that thread/daemon back to do other tasks
        
q = Queue()

for x in range(10):
    # to make 10 threads
    t = threading.Thread(target = threader)
    t.daemon = True # so it dies when the main thread dies
    t.start()

start = time.time()

for worker in range(20):
    # 20 jobs to be done by 10 workers
    q.put(worker)
    
q.join()

print('Entire job took:',time.time()-start)

Thread-28 2
Thread-29 3
Thread-31 5
Thread-34 9
Thread-30 4
Thread-35 8
Thread-33 7
Thread-32 6
Thread-27 1
Thread-26 0
Thread-28 10
Thread-29 11
Thread-31 12
Thread-34 13
Thread-30 14
Thread-35 15
Thread-33 16
Thread-32 17
Thread-27 18
Thread-26 19
Entire job took: 1.012166976928711


```python
t = threading.Thread(target=threader) 
t.start()
```
This creates different threads and tell them to execute a command called "threader" (confusing name, but he could have called it "banana"), and puts it in an object called t. But first, you have to start the thread by using the `<thread object>.start()` command 

Since this runs on a loop 10 times - it means 10 different threads are created and are executing "threader"/"banana" - command (and this is the important part) simultaneously! (!!!) Meaning, the 2nd loop thread does not wait until the first loop thread finishes before it executes - it runs off to do the command side by side. 

```python
t.daemon = True
```

is defining the thread as a daemon, meaning it is not the main thread. I don't see the significance of that in this program, and maybe you can take it off. 

Now what does the threader command/function do?
First - it gives the thread an index from 0-19, and then it tells him to do exampleJob, which is just to wait 1/2 a second, and print the thread name, and the index. 

```python
print_lock = threading.Lock()
with print_lock:
    print...
```
are used to lock the other threads from stepping on each other while printing. Meaning - if thread #9 is now printing, all the other threads that currently arrived at this command have to wait until it finishes. 

So you will get printed a random arrangement of the 10 different threads names, and the "index" they were executing. Each thread will get 2 "indexes", resulting in 20 instances of the printed command.  

This (above) is the important code for threading. Down below is using the queue code 

Why 2? Why 20?

```python
for worker in range(20):
    q.put(worker)
```
    
the reason it will happen 20 times, is because there are 20 "indexes" being sent to the threading/banana command, and-

```python
worker = q.get()
exampleJob(worker)
```
The ten threads created are reaching here basically at the same time, are assigned an "index", and execute exampleJob with that index (here "worker" is actually a new local variable, that gets the other "worker" variable out of the main code, again a bit confusing, but he could have called it a different name). This is looped until-

```python
q.task_done()
q.join()
```
All the "indexes" are "returned". 

Then the program continues to its last command (print...) and finishes. Since there are 20 "indexes", there will be 20 times exampleJob will be executed, but since there are 10 threads, they will do simultaneously the first 10 exampleJob, come back and do the second 10 exampleJob (this is why, "index"-wise, the first 10 indexes will always be 0-9, and the last 10 indexes will always be 10-19). 

Since every time the 10 threads execute exampleJob they have to wait 1/2 a second, and they do this twice, the whole thing takes around 1 second, in total.

# PyMoondra

https://www.youtube.com/watch?v=2ZwuKeL0aHs&list=PLGKQkV4guDKEv1DoK4LYdo2ZPLo6cyLbm
## Single thread making example

In [10]:
import threading
import time

def sleeper(n, name):
    print(f'Hi, I am {name}, going to sleep for {n} secs.\n')
    time.sleep(n)
    print(f'{name} has woken up from sleep.\n')

# Need to call the Thread() class
# target parameter needs a function
# name is just to give a unique name to this thread
# args for the function specified by target
# t.start() to actually start the thread
t = threading.Thread(target=sleeper, name='thread1', args = (5, 'Thread #1'))

t.start()

print('This will print even before "t" finishes its task.')
print('So the main thread carries on while threading works.')
print('This is called "concurrency"\n')

t.join() # this actually starts the thread
print('t.join() (aka blocking call), blocks the interpreter from accessing the main thread till t finishes')

Hi, I am Thread #1, going to sleep for 5 secs.

This will print even before "t" finishes its task.
So the main thread carries on while threading works.
This is called "concurrency"

Thread #1 has woken up from sleep.

t.join() (aka blocking call), blocks the interpreter from accessing the main thread till t finishes


## Multiple threads that run concurrently

In [8]:
import threading
import time

def sleeper(n, name):
    print(f'Hi, I am {name}, going to sleep for {n} secs.')
    time.sleep(n)
    print(f'{name} has woken up from sleep.')

threads_list = []

start = time.time()

for i in range(5):
    t = threading.Thread(target = sleeper, 
                         name = f'Thread #{i}', 
                         args = (5, f'Thread #{i}'))
    threads_list.append(t)
    t.start() 
    print(f'{t.name} has started.\n')

for t in threads_list:
    # make sure all 5 threads end before we move back to main thread
    t.join()
    print(f'{t} has joined.\n')

end = time.time()

print(f'\nTime taken: {end-start}')

print('All 5 threads have finished their jobs.')

Hi, I am Thread #0, going to sleep for 5 secs.
Thread #0 has started.

Hi, I am Thread #1, going to sleep for 5 secs.
Thread #1 has started.

Hi, I am Thread #2, going to sleep for 5 secs.
Thread #2 has started.

Hi, I am Thread #3, going to sleep for 5 secs.
Thread #3 has started.

Hi, I am Thread #4, going to sleep for 5 secs.
Thread #4 has started.

Thread #0 has woken up from sleep.
<Thread(Thread #0, stopped 7004)> has joined.

Thread #1 has woken up from sleep.
<Thread(Thread #1, stopped 11748)> has joined.

Thread #2 has woken up from sleep.
<Thread(Thread #2, stopped 1880)> has joined.

Thread #3 has woken up from sleep.
<Thread(Thread #3, stopped 3296)> has joined.

Thread #4 has woken up from sleep.
<Thread(Thread #4, stopped 11260)> has joined.


Time taken: 5.009905099868774
All 5 threads have finished their jobs.


## Locks
https://www.youtube.com/watch?v=8BMPW49DadA&list=PLGKQkV4guDKEv1DoK4LYdo2ZPLo6cyLbm&index=6

use the `with` statement to lock the stuff, then as we know, it auto unlocks the stuff once its use is over.

`lock = threading.Lock()` to make a lock using the Lock class.

use `with lock:` then whatever variables/functions you write inside, never 2 simultaneous threads can use it at once.

without `with`, you'd use `lock.acquire()` and `lock.release()`

In [5]:
import threading
help(threading.Thread.join)

Help on function join in module threading:

join(self, timeout=None)
    Wait until the thread terminates.
    
    This blocks the calling thread until the thread whose join() method is
    called terminates -- either normally or through an unhandled exception
    or until the optional timeout occurs.
    
    When the timeout argument is present and not None, it should be a
    floating point number specifying a timeout for the operation in seconds
    (or fractions thereof). As join() always returns None, you must call
    isAlive() after join() to decide whether a timeout happened -- if the
    thread is still alive, the join() call timed out.
    
    When the timeout argument is not present or None, the operation will
    block until the thread terminates.
    
    A thread can be join()ed many times.
    
    join() raises a RuntimeError if an attempt is made to join the current
    thread as that would cause a deadlock. It is also an error to join() a
    thread before it has b

In [9]:
import queue
help(queue.Queue.join)

Help on function join in module queue:

join(self)
    Blocks until all items in the Queue have been gotten and processed.
    
    The count of unfinished tasks goes up whenever an item is added to the
    queue. The count goes down whenever a consumer thread calls task_done()
    to indicate the item was retrieved and all work on it is complete.
    
    When the count of unfinished tasks drops to zero, join() unblocks.

